Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default
```

# Define Strategy

In [4]:
%matplotlib inline
from punisher.common import *

from punisher.strategies.strategy import Strategy

class SimpleStrategy(Strategy):
    def __init__(self):
        super().__init__()

    def log_all(self, orders, data, ctx, time_utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(time_utc)
            self.log_ohlcv(data)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        orders = []
        asset = Asset(c.XRP, c.BTC)
        price = data['close']
        quantity = 1.0

        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, asset, price, quantity)
        else:
            order = order_manager.build_limit_sell_order(
                ctx.exchange, asset, price, quantity)

        orders.append(order)

        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)

        self.log_all(orders, data, ctx, data['time_utc'])
        return orders

In [5]:
strategy = SimpleStrategy()

# Backtest (Default config)

In [ ]:
runner.backtest('jupyter-demo-backtest', strategy)

# Backtest (Using Context)

In [ ]:
from punisher.trading.context import default_config, Context, TradingMode

config = default_config(TradingMode.BACKTEST)
context = Context.from_config(config)
runner.backtest('jupyter-demo-backtest', strategy, context=context)

# Simulate Orders (default config)

In [ ]:
runner.simulation('jupyter-demo-sim', strategy)

# Live Trade

In [6]:
from datetime import datetime, timedelta
base = c.XRP
quote = c.BTC
asset = Asset(base, quote)
period = Timeframe.ONE_MIN
exchange_id = c.BINANCE

config = {
        'experiment': 'default-live-demo',
        'cash_asset': c.BTC,
        'starting_cash': 0.0,
        'store': FILE_STORE,
        'balance': Balance(starting_cash=0.0).to_dict(),
        'feed': {
            'fpath': ohlcv.get_price_data_fpath(asset, exchange_id, period.value['id']),
            'start': datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S"),
            'symbols': ['XRP/BTC'],
            'timeframe': Timeframe.ONE_MIN.name,
        },
        'exchange': {
            "exchange_id": exchange_id
        }
    }

config['feed']['name'] = EXCHANGE_FEED

 # must define your own config to live trade - so you know how it works :)
runner.live('jupyter-demo-live', strategy, config)

Loading feed: /Users/cfortuner/Dropbox/punisher/binance_XRP_BTC_1m.csv
Downloading: XRP/BTC


Starting Live trading ... CAREFUL!
{'experiment': 'default-live-demo', 'cash_asset': 'BTC', 'starting_cash': 0.0, 'store': 'CSV_STORE', 'balance': {'BTC': {'free': 0.0, 'used': 0.0, 'total': 0.0}, 'free': {'BTC': 0.0}, 'used': {'BTC': 0.0}, 'total': {'BTC': 0.0}}, 'feed': {'fpath': '/Users/cfortuner/Dropbox/punisher/binance_XRP_BTC_1m.csv', 'start': '2018-01-16T05:10:35', 'symbols': ['XRP/BTC'], 'timeframe': 'ONE_MIN', 'name': 'EXCHANGE_FEED'}, 'exchange': {'exchange_id': 'binance', 'data_provider': <punisher.data.provider.PaperExchangeDataProvider object at 0x1115b6278>, 'balance': {'BTC': {'free': 0.0, 'used': 0.0, 'total': 0.0}, 'free': {'BTC': 0.0}, 'used': {'BTC': 0.0}, 'total': {'BTC': 0.0}}}}


Downloaded rows: 0
Downloading: XRP/BTC
Downloaded rows: 0
No data after prior poll: 2018-01-16 05:09:35
TOTAL VALUE
0.0
Downloading: XRP/BTC
Downloaded rows: 0
No data after prior poll: 2018-01-16 05:09:35
TOTAL VALUE
0.0
Downloading: XRP/BTC
Downloaded rows: 0
No data after prior poll: 2018-01-16 05:09:35
TOTAL VALUE
0.0
Downloading: XRP/BTC
Downloaded rows: 0
No data after prior poll: 2018-01-16 05:09:35
TOTAL VALUE
0.0
Downloading: XRP/BTC
Downloaded rows: 0
No data after prior poll: 2018-01-16 05:09:35
TOTAL VALUE
0.0
Downloading: XRP/BTC
Downloaded rows: 0
No data after prior poll: 2018-01-16 05:09:35
TOTAL VALUE
0.0
Downloading: XRP/BTC
Downloaded rows: 0
No data after prior poll: 2018-01-16 05:09:35
TOTAL VALUE
0.0
Downloading: XRP/BTC
Downloaded rows: 0
No data after prior poll: 2018-01-16 05:09:35
TOTAL VALUE
0.0
Downloading: XRP/BTC
Downloaded rows: 0
No data after prior poll: 2018-01-16 05:09:35
TOTAL VALUE
0.0
Downloading: XRP/BTC
Downloaded rows: 0
No data after prior pol

---------------------------------------
Epoch 1 - Timestep: 2018-01-16T05:11:00
---------------------------------------
OHLCV
    O: 0.0001 | C: 0.0001 | V: 222.0 | T: 2018-01-16T05:11:00
ORDERS
    1: XRP/BTC | LIMIT_SELL | Price: 0.0001 | Quantity: 1.0000
PERFORMANCE
    Cash: 0.0000 Total Val: 0.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.03307101, 'used': 0.0, 'total': 0.03307101}
    LTC - {'free': 0.0, 'used': 0.0, 'total': 0.0}
    ETH - {'free': 0.0, 'used': 0.0, 'total': 0.0}
    BNC - {'free': 0.0, 'used': 0.0, 'total': 0.0}
    ICO - {'free': 0.0, 'used': 0.0, 'total': 0.0}
    NEO - {'free': 0.0, 'used': 0.0, 'total': 0.0}
    BNB - {'free': 0.0, 'used': 0.0, 'total': 0.0}
    123 - {'free': 0.0, 'used': 0.0, 'total': 0.0}
    456 - {'free': 0.0, 'used': 0.0, 'total': 0.0}
    QTUM - {'free': 0.0, 'used': 0.0, 'total': 0.0}
    EOS - {'free': 0.0, 'used': 0.0, 'total': 0.0}
    SNT - {'free': 0.0, 'used': 0.0, 'total': 0.0}
    BNT - {'free': 0.0, 'used': 

TypeError: 'NoneType' object is not iterable